In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
rating_df = pd.read_csv('../input/anime-recommendations-database/rating.csv')
anime_df = pd.read_csv('../input/anime-recommendations-database/anime.csv')

In [ ]:
anime_df.head(20)

In [ ]:
rating_df[150:170]

In [ ]:
m_user = max(rating_df['user_id'])
m_anime = max(rating_df['anime_id'])
rating_df.loc[42]

In [ ]:
L = [[] for k in range(m_user)]
for k in rating_df.index[:100000]:
    ligne = rating_df.loc[k]
    user, anime, note = ligne[0], ligne[1], ligne[2]
    L[user - 1].append((anime, note))
L

In [ ]:
def rating_user(l):
        anime = [k[0] for k in l]
        notes = [k[1] for k in l]
        h = [k for k in notes if k != -1]
        if len(h) !=0:
            moy = np.mean(h)
            table = []
            for k in range(len(notes)):
                if notes[k] ==-1:
                    table.append(3/4*moy + 1) #idée : / moy * 10 ?
                else:
                    table.append(notes[k] + 1)
            return table
        else :
            table = []
            for k in range(len(notes)):
                table.append(6)
            return table
        return []

In [ ]:
rating_user(L[0])

In [ ]:
new_rating = []
for i in L:
    new_rating += rating_user(i)
new_rating

In [ ]:
A=[]
for k in L:
    for i in k:
        A.append(i[0])
A
    

In [ ]:
anime_vues = []
for k in L:
    vues = []
    for i in k:
        vues.append(i[0])
    anime_vues.append(vues)
anime_vues

In [ ]:
popular_df_transitoire = pd.DataFrame({'anime_id': A,'note': new_rating})

In [ ]:
popular_df_transitoire.head(10)

In [ ]:
anime_popularity_df = popular_df_transitoire.groupby('anime_id')['note'].sum().sort_values(ascending=False).reset_index()
anime_popularity_df.head(10)

In [ ]:
anime_df.loc[anime_df['anime_id'] == 20]

In [ ]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore, topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['anime_id'].isin(items_to_ignore)] \
                               .sort_values('note', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'anime_id', 
                                                          right_on = 'anime_id')[['anime_id', 'name','rating','note']]


        return recommendations_df
    
popularity_model = PopularityRecommender(anime_popularity_df, anime_df)

In [ ]:
popularity_model.recommend_items(3, anime_vues[2], topn=20, verbose=True)